# Tool recommendation 
## (Random forest with weighted cross-entropy loss)

In [14]:
import numpy as np
import json
import warnings
import operator

import h5py
from joblib import load

warnings.filterwarnings("ignore")


def read_file(file_path):
    with open(file_path, 'r') as data_file:
        data = json.loads(data_file.read())
    return data


def create_model(model_path):
    loaded_model = load(model_path) 
    return loaded_model


def get_predicted_tools(base_tools, predictions, topk):
    """
    Get predicted tools. If predicted tools are less in number, combine them with published tools
    """
    intersection = list(set(predictions).intersection(set(base_tools)))
    return intersection[:topk]


def sort_by_usage(t_list, class_weights, d_dict):
    """
    Sort predictions by usage/class weights
    """
    tool_dict = dict()
    for tool in t_list:
        t_id = d_dict[tool]
        tool_dict[tool] = class_weights[str(t_id)]
    tool_dict = dict(sorted(tool_dict.items(), key=lambda kv: kv[1], reverse=True))
    return list(tool_dict.keys()), list(tool_dict.values())


def separate_predictions(base_tools, predictions, last_tool_name, weight_values, topk):
    """
    Get predictions from published and normal workflows
    """
    last_base_tools = list()
    predictions = predictions * weight_values
    prediction_pos = np.argsort(predictions, axis=-1)
    topk_prediction_pos = prediction_pos[-topk:]
    # get tool ids
    pred_tool_ids = [reverse_dictionary[str(tool_pos)] for tool_pos in topk_prediction_pos]
    if last_tool_name in base_tools:
        last_base_tools = base_tools[last_tool_name]
        if type(last_base_tools).__name__ == "str":
            # get published or compatible tools for the last tool in a sequence of tools
            last_base_tools = last_base_tools.split(",")
    # get predicted tools
    p_tools = get_predicted_tools(last_base_tools, pred_tool_ids, topk)
    sorted_c_t, sorted_c_v = sort_by_usage(p_tools, class_weights, dictionary)
    return sorted_c_t, sorted_c_v


def compute_recommendations(model, tool_sequence, labels, dictionary, reverse_dictionary, class_weights, topk=10, max_seq_len=25):
    tl_seq = tool_sequence.split(",")
    tl_seq_ids = [str(dictionary[t]) for t in tl_seq]
    last_tool_name = tl_seq[-1]
    sample = np.zeros(max_seq_len)
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    for idx, tool_id in enumerate(tl_seq_ids):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))
    tool_sequence_names = [reverse_dictionary[str(tool_pos)] for tool_pos in tl_seq_ids]
    # predict next tools for a test path
    prediction = model.predict(sample_reshaped)
    nw_dimension = prediction.shape[1]
    prediction = np.reshape(prediction, (nw_dimension,))
    
    half_len = int(nw_dimension / 2)
    
    pub_t, pub_v = separate_predictions(standard_connections, prediction[:half_len], last_tool_name, weight_val, topk)
    # get recommended tools from normal workflows
    c_t, c_v = separate_predictions(compatible_tools, prediction[half_len:], last_tool_name, weight_val, topk)
    # combine predictions coming from different workflows
    # promote recommended tools coming from published workflows
    # to the top and then show other recommendations
    print()
    tool_seq_name = ",".join(tool_sequence_names)
    print("Current tool sequence: ")
    print()
    print(tool_seq_name)
    print()
    print("Overall recommendations: ")
    print()
    pub_t.extend(c_t)
    pub_v.extend(c_v)
    # remove duplicates if any
    pub_t = list(dict.fromkeys(pub_t))
    pub_v = list(dict.fromkeys(pub_v))
    print(pub_t)
    ids_tools = dict()
    for key in pub_t:
        ids_tools[key] = dictionary[key]
    print()
    print("Recommended tool ids:")
    print()
    for i in ids_tools:
        rev_id = dictionary[i]
        wt = class_weights[str(rev_id)]
        print(i + "(" + str(ids_tools[i]) + ")" + "(" + str(wt) + ")")

## Unpack trained model for prediction

In [15]:
output_path = "data/tool_recommendation_model.hdf5"
model_path = "data/model.joblib"

output_model = h5py.File(output_path, 'r')
dictionary = json.loads(output_model.get('data_dictionary').value)
reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
class_weights = json.loads(output_model.get('class_weights').value)
standard_connections = json.loads(output_model.get('standard_connections').value)
compatible_tools = json.loads(output_model.get('compatible_tools').value)
best_parameters = json.loads(output_model.get('best_parameters').value)
loaded_model = create_model(model_path)

## Example tools

In [16]:
# bowtie2
# trimmomatic

## Recommended tools

In [21]:
topk = 10 # set the maximum number of recommendations

# Give tools ids in a sequence and see the recommendations. # To know all the tool ids, 
# please print the variable 'reverse_dictionary'
tool_seq = "Cut1"

compute_recommendations(loaded_model, tool_seq, "", dictionary, reverse_dictionary, class_weights, topk)


Current tool sequence: 

Cut1

Overall recommendations: 

['addValue', 'plotly_ml_performance_plots']

Recommended tool ids:

addValue(517)(5.766569)
plotly_ml_performance_plots(515)(2.892838)


In [22]:
best_parameters

{'criterion': 'entropy',
 'max_depth': 7,
 'max_features': None,
 'min_samples_split': 0.0012386307312684402,
 'n_estimators': 15}